# Облагораживаем код

Задача: Написать код кормления живтных и кормушки. Есть два вида животных Кот и Собака. Кормушка может быть в открытом и закрытом положениях. Собака умеет открывать кормушку, а также может довольствоваться остатками. Кошка всегда требует фикисровнное количество пищи.




* abc
* enum
* typing
* context
* observable
* factory


## Структурная типизация

### Модуль `typing`

### Протоколы

### MyPy

## Декораторы

* [Понимаем декораторы в Python'e, шаг за шагом. Шаг 1](https://habr.com/ru/post/141411/)
* [Понимаем декораторы в Python'e, шаг за шагом. Шаг 2](https://habr.com/ru/post/141501/)
* [Шаблон проектирования **Декоратор**](https://ru.wikipedia.org/wiki/%D0%94%D0%B5%D0%BA%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80_(%D1%88%D0%B0%D0%B1%D0%BB%D0%BE%D0%BD_%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F))

In [1]:
def my_shiny_new_decorator(a_function_to_decorate):

    def the_wrapper_around_the_original_function():
        print("Я - код, который отработает до вызова функции")
        a_function_to_decorate()
        print("А я - код, срабатывающий после")

    return the_wrapper_around_the_original_function

In [2]:
def a_stand_alone_function():
    print("Я простая одинокая функция, ты ведь не посмеешь меня изменять?..")

a_stand_alone_function()

Я простая одинокая функция, ты ведь не посмеешь меня изменять?..


In [3]:
a_stand_alone_function_decorated = my_shiny_new_decorator(a_stand_alone_function)

a_stand_alone_function_decorated()

Я - код, который отработает до вызова функции
Я простая одинокая функция, ты ведь не посмеешь меня изменять?..
А я - код, срабатывающий после


In [ ]:
@my_shiny_new_decorator
def another_stand_alone_function():
    print("Оставь меня в покое")

another_stand_alone_function()

### Можно применять несколько декораторов, порядок применения важен

In [4]:
def bread(func):
    def wrapper():
        print("</------\>")
        func()
        print("<\______/>")
    return wrapper

def ingredients(func):
    def wrapper():
        print("#помидоры#")
        func()
        print("~салат~")
    return wrapper

In [7]:
@bread
@ingredients
def sandwich(food="--ветчина--"):
    print(food)

sandwich()

</------\>
#помидоры#
--ветчина--
~салат~
<\______/>


In [8]:
@ingredients
@bread
def sandwich(food="--ветчина--"):
    print(food)

sandwich()

#помидоры#
</------\>
--ветчина--
<\______/>
~салат~


### Передача («проброс») аргументов в декорируемую функцию

In [9]:
def a_decorator_passing_arguments(function_to_decorate):
    def a_wrapper_accepting_arguments(arg1, arg2):
        print("Смотри, что я получил:", arg1, arg2)
        function_to_decorate(arg1, arg2)
    return a_wrapper_accepting_arguments

@a_decorator_passing_arguments
def print_full_name(first_name, last_name):
    print("Меня зовут", first_name, last_name)

print_full_name("Питер", "Венкман")

Смотри, что я получил: Питер Венкман
Меня зовут Питер Венкман


### Декораторы методов

In [10]:
def method_friendly_decorator(method_to_decorate):
    def wrapper(self, feeder):
        feeder.food -= 10
        return method_to_decorate(self, feeder)
    return wrapper


class Feeder(object):

    def __init__(self):
        self.food = 60

class Cat:
    def __init__(self):
        self.food_intake = 20

    @method_friendly_decorator
    def rob_feeder(self, feeder):
        feeder.food -= self.food_intake

feeder = Feeder()
Cat().rob_feeder(feeder)
print(feeder.food)

30


### Фабрика декораторов

In [15]:
def decorator_maker():
    print("Я создаю декораторы! Я буду вызван только раз: "
          "когда ты попросишь меня создать тебе декоратор.")

    def my_decorator(func):
        print("Я - декоратор! Я буду вызван только раз: в момент декорирования функции.")

        def wrapped():
            print ("Я - обёртка вокруг декорируемой функции. "
                  "Я буду вызвана каждый раз когда ты вызываешь декорируемую функцию. "
                  "Я возвращаю результат работы декорируемой функции.")
            return func()

        print("Я возвращаю обёрнутую функцию.")

        return wrapped

    print("Я возвращаю декоратор.")
    return my_decorator

In [16]:
@decorator_maker()
def decorated_function():
    print("Я - декорируемая функция.")

Я создаю декораторы! Я буду вызван только раз: когда ты попросишь меня создать тебе декоратор.
Я возвращаю декоратор.
Я - декоратор! Я буду вызван только раз: в момент декорирования функции.
Я возвращаю обёрнутую функцию.


In [17]:
decorated_function()

Я - обёртка вокруг декорируемой функции. Я буду вызвана каждый раз когда ты вызываешь декорируемую функцию. Я возвращаю результат работы декорируемой функции.
Я - декорируемая функция.


In [19]:
def decorator_maker_with_arguments(decorator_arg1, decorator_arg2):

    print("Я создаю декораторы! И я получил следующие аргументы:", decorator_arg1, decorator_arg2)

    def my_decorator(func):
        print("Я - декоратор. И ты всё же смог передать мне эти аргументы:", decorator_arg1, decorator_arg2)

        # Не перепутайте аргументы декораторов с аргументами функций!
        def wrapped(function_arg1, function_arg2) :
            print ("Я - обёртка вокруг декорируемой функции.\n"
                  "И я имею доступ ко всем аргументам: \n"
                  "\t- и декоратора: {0} {1}\n"
                  "\t- и функции: {2} {3}\n"
                  "Теперь я могу передать нужные аргументы дальше"
                  .format(decorator_arg1, decorator_arg2,
                          function_arg1, function_arg2))
            return func(function_arg1, function_arg2)

        return wrapped

    return my_decorator

@decorator_maker_with_arguments("Леонард", "Шелдон")
def decorated_function_with_arguments(function_arg1, function_arg2):
    print ("Я - декорируемая функция и я знаю только о своих аргументах: {0}"
           " {1}".format(function_arg1, function_arg2))

decorated_function_with_arguments("Раджеш", "Говард")

Я создаю декораторы! И я получил следующие аргументы: Леонард Шелдон
Я - декоратор. И ты всё же смог передать мне эти аргументы: Леонард Шелдон
Я - обёртка вокруг декорируемой функции.
И я имею доступ ко всем аргументам: 
	- и декоратора: Леонард Шелдон
	- и функции: Раджеш Говард
Теперь я могу передать нужные аргументы дальше
Я - декорируемая функция и я знаю только о своих аргументах: Раджеш Говард


In [22]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло
    выполнение декорируемой функции.
    """
    import time
    def wrapper(*args, **kwargs):
        t = time.clock()
        res = func(*args, **kwargs)
        print(func.__name__, time.clock() - t)
        return res
    return wrapper


def logging(func):
    """
    Декоратор, логирующий работу кода.
    (хорошо, он просто выводит вызовы, но тут могло быть и логирование!)
    """
    def wrapper(*args, **kwargs):
        res = func(*args, **kwargs)
        print(func.__name__, args, kwargs)
        return res
    return wrapper


def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов
    декорируемой функции.
    """
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        res = func(*args, **kwargs)
        print("{0} была вызвана: {1}x".format(func.__name__, wrapper.count))
        return res
    wrapper.count = 0
    return wrapper


@benchmark
@logging
@counter
def reverse_string(string):
    return str(reversed(string))

print(reverse_string("А роза упала на лапу Азора"))
print(reverse_string("A man, a plan, a canoe, pasta, heros, rajahs, a coloratura, maps, snipe, percale, macaroni, a gag, a banana bag, a tan, a tag, a banana bag again (or a camel), a crepe, pins, Spam, a rut, a Rolo, cash, a jar, sore hats, a peon, a canal: Panama!"))


AttributeError: module 'time' has no attribute 'clock'

* Декораторы могут быть использованы для расширения:
    * возможностей функций и классов;
    * одним и тем же кодом.
*
* Декоратор вызывается ровно один раз.
* Когда мы пишем `import x`"` все функции из `x` декорируются сразу же, и мы уже не сможем ничего изменить.

### Популярные декораторы

## Модульное тестирование
